In [9]:
from flask import Flask, request, jsonify
import pickle
import pandas as pd

In [10]:
app = Flask(__name__)

In [11]:
with open('decision_tree_model.pkl', 'rb') as file:
    dt_reg = pickle.load(file)

# Load the MinMaxScaler from the saved file
with open('price_scaler.pkl', 'rb') as file:
    price_scaler = pickle.load(file)

with open('Engine_scaler.pkl', 'rb') as file:
   Engine_scaler = pickle.load(file)

In [12]:
csv_file_path = 'C:\\Users\\hp\\OneDrive\\Desktop\\assM\\cleaned_data.csv'

In [13]:
training_data = pd.read_csv(csv_file_path)

In [14]:
excluded_columns = ['Price'] 
feature_names = [col for col in training_data.columns if col not in excluded_columns]

In [15]:
@app.route('/predict', methods=['POST'])
def predict():
    try:
        # Initialize DataFrame with default values zeros
        default_data = pd.DataFrame([[0] * len(feature_names)], columns=feature_names)

        data = request.get_json(force=True)
        app.logger.info(f"Received data: {data}")

        # Convert received data to DataFrame
        input_data = pd.DataFrame([data])
        # Extract features used during training
        features_used_during_training = input_data.columns
        input_data_selected = input_data[features_used_during_training]
        default_data.update(input_data_selected)

        # Scale the 'Motor power' feature using the motor scaler
        default_data['Engine Power'] = Engine_scaler.transform(default_data[['Engine Power']])

        # Make predictions using the model
        predictions = dt_reg.predict(default_data)
        # Inverse transform the normalized predictions
        original_scale_predictions = price_scaler.inverse_transform(predictions.reshape(-1, 1))
        app.logger.info(f"Predicting with data: {default_data}")

        return jsonify({'predicted_price': original_scale_predictions.tolist()})

    except Exception as e:
        app.logger.error(f"Error: {str(e)}")
        return jsonify({'error': f'Internal Server Error: {str(e)}'})

In [16]:
if __name__ == '__main__':
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [09/Dec/2023 19:42:11] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:42:22] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:44:53] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:46:27] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:50:09] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:50:16] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:51:13] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:53:00] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:55:32] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:57:22] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:58:05] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [09/Dec/2023 19:58:47] "POST /predict HTTP/1.1" 200 -
